# 2D Detector Geometry

Various issues come up when calibrating 2D detector geometry and doing strain refinements:

- Numerical instabilities can arise for small angles in certain formulations

- Exact answers for angles like 30,45,60,90,120 etc are hard to reproduce

- ...leading to some philosophical misgivings about trigonometric functions

- People want to get strains that are independent of orientation (despite JW resisting this idea for years)

- There is a correlation between wavelength and distance (not addressed here)

- Youtube videos from Norman J. Wildberger suggested looking a quadrance (length^2) and spread (sin^2 angle). He has a book "DIVINE PROPORTIONS : Rational Trigonometry to Universal Geometry" Wild Egg Books, Sydney 2005. ISBN: 0-9757492-0-X. It might be useful to get a copy and try to read it.

We set up the geometry to be ready to work on data where the detector was moved. This happens to follow the fable convention and it is a bit more common than moving the X-ray beam (but that obviously happens too). Anyway, the detector itself doest not define the geometry here. We place the x-axis along the X-ray beam. The detector is wherever it is, and some function gives the x,y,z co-ordinates of each pixel. A photon is detected at pixel co-ordinates $(x,y,z)$ in 3D cartesian space, usually in the sensitive surface of the detector.

The origin of the coordinate system is usually in the centre of the sample. If the sample is not at the origin, then subtract the sample coordinates from the detector coordinates to get a vector along the direction of the output ray below.

The goal here is to get a formula for the metric tensor in terms of the pixel co-ordinates. Elsewhere we have formulae for the various finite strain tensors in terms of the metric tensor.

Consider the cross product of vectors along the incident and scattered rays. This is:

$ \mathbf{s_0} \times \mathbf{s_1} = (1,0,0) \times (x,y,z) = |s_0|  |s_1| \sin2\theta \mathbf{n} $

$ \mathbf{s_0} \times \mathbf{s_1} = (0,z,-y) = (1).(\sqrt{x^2+y^2+z^2}) \sin2\theta \mathbf{n} $

...where $\mathbf{n}$ is a unit vector normal to the incident and scattered rays. We dot this vector with itself to arrive at a scalar value:

$ (\mathbf{s_0} \times \mathbf{s_1}).(\mathbf{s_0} \times \mathbf{s_1})
= (y^2 + z^2) = (x^2 + y^2 + z^2) \sin^22\theta $

This gives us a relationship between $\sin^22\theta$ and the detector coordinates. From Bragg's law, $\lambda = 2d\sin\theta$, we can find the metric tensor in terms of $\sin^2\theta$, (note the $\theta$ versus $2\theta$):

$ \mathbf{ h g h^T } = \frac{1}{d^2} = \frac{ 4 \sin^2\theta } {\lambda^2} $

... where $\mathbf{g}$ is the reciprocal metric tensor. 

We convert the $2\theta$ into $\theta$ via trig identities:

$ \sin^22\theta = (2\sin\theta\cos\theta)^2 = 4 \sin^2\theta \cos^2\theta = 4 \sin^2\theta (1 - \sin^2\theta)$

This gives us a quadratic in $\sin^2\theta$:

$ (\sin^2\theta)^2 - (\sin^2\theta) + \frac { (y^2 + z^2) } {4 (x^2 + y^2 + z^2) } = 0 $

This can be solved using a quadratic formula for $ax^2+bx+c$ that avoids roundoff for small scattering angles where $c$ approaches zero (https://web.physics.utah.edu/~detar/lessons/python/quadratic/node3.html):

$ x = \frac{2.c}{(-b \mp \sqrt( b^2 - 4ac )} $

We set, $ R = z^2 + y^2 $ and $ Q = x^2 + y^2 + z^2 $ to get:

$ Q (\sin^2\theta)^2 - Q (\sin^2\theta) + R/4 = 0 $

$ \sin^2\theta = \frac{R}{2 ( Q \mp \sqrt{ Q^2 - Q R } ) } $

Note that $ Q = R + x^2 $:

$ \sin^2\theta = \frac{R}{2 ( Q \mp \sqrt{ Q (R + x^2) - Q R })} = \frac{R}{2 ( Q \mp x\sqrt{ Q })} $

$  \mathbf{ h g h^T } = \frac{1}{d^2} = \frac{ 4 \sin^2\theta } {\lambda^2} =  \frac{2 R}{ \lambda^2 ( Q \mp x\sqrt{ Q } ) } $

In [1]:
from numpy import arctan2, sin, cos, sqrt, allclose, where, isfinite

def sinth2_atan( x, y, z ):
    """ Compute sin(theta)^2
    Conventional approach in ImageD11 now
    """
    twotheta = arctan2( sqrt(z*z + y*y), x )
    sinth = sin( twotheta/2 )
    return sinth**2

def sinth2_sqrt( x, y, z ):
    """ Compute sin(theta)^2 
    x,y,z = co-ordinates of the pixel in cartesian space
    R = hypotenuse normal to incident beam (along x)
    Q = hypotenuse along the scattered beam
    """
    R = y*y+z*z
    Q = x*x + R
    sinsqth = 0.5*R/( Q + x*sqrt(Q) ) #  postive root only, not 0.5*R/( Q - x*sqrt(Q) )
    return sinsqth

In [2]:
tests = [(0,1000,1000),       # two theta == 90 degrees, sin(45)=1/sqrt(2), ans=0.5
              (1, sqrt(3), 0),     # two theta == 60 degrees, sin(30)=1/2, ans=0.25
              (1e-9,1000,1000), 
              (1e9,0,1),
              (1e20,1,0),
              (100,0,0),
              (1000,1000,1000),
                       (-123,456,789)]

for x,y,z in tests:
    v1 = sinth2_atan(x,y,z)            
    v2 = sinth2_sqrt(x,y,z)
    e = v1-v2
    print(e,x,y,z,v1,v2)
    assert allclose( v1, v2)

-1.1102230246251565e-16 0 1000 1000 0.4999999999999999 0.5
-5.551115123125783e-17 1 1.7320508075688772 0 0.24999999999999994 0.25
5.551115123125783e-17 1e-09 1000 1000 0.4999999999996465 0.49999999999964645
0.0 1000000000.0 0 1 2.5e-19 2.5e-19
0.0 1e+20 1 0 2.5e-41 2.5e-41
0.0 100 0 0 0.0 0.0
-2.7755575615628914e-17 1000 1000 1000 0.2113248654051871 0.21132486540518713
1.1102230246251565e-16 -123 456 789 0.5668799937442661 0.566879993744266


In [3]:
# diversion : derivatives of the atan expression:
"""
[jon@pine docs]$ maxima 
;;; Loading #P"/usr/lib/ecl-21.2.1/sb-bsd-sockets.fas"
;;; Loading #P"/usr/lib/ecl-21.2.1/sockets.fas"
Maxima 5.44.0 http://maxima.sourceforge.net
using Lisp ECL 21.2.1
Distributed under the GNU Public License. See the file COPYING.
Dedicated to the memory of William Schelter.
The function bug_report() provides bug reporting information.
(%i1) display2d : false;
(%o1) false
(%i2) s : sin( atan2( sqrt( y*y+z*z), x ) / 2 );  
(%o2) sin(atan2(sqrt(z^2+y^2),x)/2)
(%i3) diff( s,x,1); 
(%o3) -(sqrt(z^2+y^2)*cos(atan2(sqrt(z^2+y^2),x)/2))/(2*(z^2+y^2+x^2))
(%i4) diff(s,y,1);
(%o4) (x*y*cos(atan2(sqrt(z^2+y^2),x)/2))/(2*sqrt(z^2+y^2)*(z^2+y^2+x^2))
(%i5) diff(s,z,1);    
(%o5) (x*z*cos(atan2(sqrt(z^2+y^2),x)/2))/(2*sqrt(z^2+y^2)*(z^2+y^2+x^2))
"""

# import numba    # no great effect here
# @numba.njit   
def sinth2_atan_deriv( x, y, z ):
    """ Compute sin(theta)**2 and derivatives w.r.t x,y,z """
    R2 = z*z+y*y
    r  = sqrt( R2 )
    D2 = R2 + x*x    # x*x+y*y+z*z
    theta = arctan2( r, x ) / 2
    sinth = sin( theta )
    costh = cos( theta )
    div_D2 = where(isfinite(1/D2),1/D2,0)
    p = sinth*costh*div_D2
    dsinth2_dx = - r*p
    # if r == 0 this blows up. For r==0 then y==0 as well. Should somehow determine y==0...
    div_r = where(isfinite(1/r), 1/r, 0)
    dsinth2_dy =  x*y*p*div_r # at r == 0?
    dsinth2_dz =  x*z*p*div_r
    return sinth*sinth, dsinth2_dx, dsinth2_dy, dsinth2_dz

In [4]:
"""# maxima:
(%i3) r:z*z+y*y
(%o3) z^2+y^2
(%i4) q:r+x*x
(%o4) z^2+y^2+x^2
(%i5) s:r/(2*(q+x*sqrt(q)))
(%o5) (z^2+y^2)/(2*(x*sqrt(z^2+y^2+x^2)+z^2+y^2+x^2))
(%i6) diff(s,x,1)
(%o6) -((z^2+y^2)*(sqrt(z^2+y^2+x^2)+x^2/sqrt(z^2+y^2+x^2)+2*x))
 /(2*(x*sqrt(z^2+y^2+x^2)+z^2+y^2+x^2)^2)
(%i7) diff(s,y,1)
(%o7) y/(x*sqrt(z^2+y^2+x^2)+z^2+y^2+x^2)
 -((z^2+y^2)*((x*y)/sqrt(z^2+y^2+x^2)+2*y))
  /(2*(x*sqrt(z^2+y^2+x^2)+z^2+y^2+x^2)^2)
(%i8) diff(s,z,1)
(%o8) z/(x*sqrt(z^2+y^2+x^2)+z^2+y^2+x^2)
 -((z^2+y^2)*((x*z)/sqrt(z^2+y^2+x^2)+2*z))
  /(2*(x*sqrt(z^2+y^2+x^2)+z^2+y^2+x^2)^2)
"""
# @numba.njit
def sinth2_sqrt_deriv(x, y, z):
    R = z*z + y*y
    Q = R + x*x
    SQ = sqrt(Q)
    R2 = R/2
    # at x==y==0 this is undefined.
    rQ_xSQ = 1/(Q + x*SQ)
    sinth2 = R2*rQ_xSQ
    # some simplification and collecting terms from expressions above to get:
    sr = sinth2*rQ_xSQ
    p = (x/SQ+2)*sr  # p should be in the range 3sr -> 2sr for x/x to 0/sqrt(R)
    t = (rQ_xSQ - p) # may cancel?  
    sinth2_dx =  -(SQ*sr+x*p)
    sinth2_dy =   y*t
    sinth2_dz =   z*t
    return sinth2, sinth2_dx, sinth2_dy, sinth2_dz

In [5]:
for x,y,z in tests:
    a1 = sinth2_sqrt_deriv(x,y,z)
    a2 = sinth2_atan_deriv(x,y,z)
    print('xyz:', x,y,z,"\nsqrt\t",a1,"\natan\t",a2)
    assert allclose(a1,a2)

xyz: 0 1000 1000 
sqrt	 (0.5, -0.00035355339059327376, 0.0, 0.0) 
atan	 (0.4999999999999999, -0.00035355339059327376, 0.0, 0.0)
xyz: 1 1.7320508075688772 0 
sqrt	 (0.25, -0.1875, 0.10825317547305482, 0.0) 
atan	 (0.24999999999999994, -0.18750000000000003, 0.10825317547305487, 0.0)
xyz: 1e-09 1000 1000 
sqrt	 (0.49999999999964645, -0.00035355339059327376, 1.768181277393352e-16, 1.768181277393352e-16) 
atan	 (0.4999999999996465, -0.00035355339059327376, 1.767766952966369e-16, 1.767766952966369e-16)
xyz: 1000000000.0 0 1 
sqrt	 (2.5e-19, -5e-28, 0.0, 5e-19) 
atan	 (2.5e-19, -5.000000000000001e-28, 0.0, 5.000000000000001e-19)
xyz: 1e+20 1 0 
sqrt	 (2.5e-41, -4.9999999999999985e-61, 5e-41, 0.0) 
atan	 (2.5e-41, -4.999999999999999e-61, 5e-41, 0.0)
xyz: 100 0 0 
sqrt	 (0.0, -0.0, 0.0, 0.0) 
atan	 (0.0, -0.0, 0.0, 0.0)
xyz: 1000 1000 1000 
sqrt	 (0.21132486540518713, -0.00019245008972987527, 9.622504486493763e-05, 9.622504486493763e-05) 
atan	 (0.2113248654051871, -0.00019245008972987527, 9.62

/tmp/ipykernel_751591/3023911578.py:37: RuntimeWarning: divide by zero encountered in double_scalars
  div_r = where(isfinite(1/r), 1/r, 0)


In [6]:
from numpy import isclose, random, arange
random.seed(11)
for s,o in [(1e6,0.5), (1,.25), (1e-6,0.5)]:
    x,y,z = (random.random( (3,1_000_000) )-o)*s
    a1 = sinth2_sqrt_deriv(x,y,z)
    a2 = sinth2_atan_deriv(x,y,z)
    for i in range(4):
        if allclose(a1[i], a2[i]):
            continue
        absdiff = abs(a1[i]-a2[i])
        e = absdiff.argmax()
        print(i,e, x[e],y[e],z[e])
        print(a1[i][e],a2[i][e])
        # The derivatives start to lose precision for small values of x,y,z:

1 629650 -4.2043328642028395e-07 -3.2161595933277716e-10 9.70597750455382e-10
-7.0338134765625 -7.033922666496423
2 54669 -3.3884238262201716e-07 5.078959545993244e-10 4.734340882978083e-10
-2211.7232267291006 -2211.8036421740667
3 629650 -4.2043328642028395e-07 -3.2161595933277716e-10 9.70597750455382e-10
-2745.3556259377906 -2745.4355099133227


In [7]:
%timeit a1 = sinth2_sqrt_deriv(x,y,z)

110 ms ± 4.27 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
%timeit a2 = sinth2_atan_deriv(x,y,z)

274 ms ± 2.25 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## conclusion

This method seems to be more accurate for angles where exact answers are known and appears to be slightly faster.

Still need to do something useful with it.

Could set up to refine the metric tensor elements directly from spot positions on the detector. Does not depend on rotation angles (except for grain origins).

There should be some way to look at orientations to go along with this.